<a href="https://colab.research.google.com/github/IOlivaresPalma/LLM-Agents/blob/main/Generar_Imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain langchain-groq transformers

In [ ]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from IPython.display import Markdown, display
from langchain_core.tools import tool
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.agents import AgentExecutor
import torch
import requests
from PIL import Image
from io import BytesIO
from matplotlib import pyplot as plt

# We'll be exploring a number of pipelines today!
from diffusers import (
    StableDiffusionPipeline,
    StableDiffusionImg2ImgPipeline,
    StableDiffusionInpaintPipeline,
    StableDiffusionDepth2ImgPipeline
    )

Se carga el modelo de Stable-difusion

In [ ]:
import os

os.environ["GROQ_API_KEY"] = "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"

In [ ]:
model_id = "stabilityai/stable-diffusion-2-1-base"

scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# Definicion de las Tools

In [ ]:
@tool
def generador_imagenes(query: str) -> str:
  '''Genera, enseña o muestra una imagen dependiendo de la frase que se entregue'''
  prompt = query

  prompt_template1 = PromptTemplate.from_template(
    "Genera contexto y descripcion para la frase siguiente: '{imagen}'"
  )

  context = prompt_template1.format(imagen=prompt)
  #print(context)

  esp = llm.invoke(context)
  print(esp.content)

  prompt_template2 = PromptTemplate.from_template(
    "Traduce el siguiente texto del idioma que detectas al ingles: '{frase}'"
  )

  espannol = prompt_template2.format(frase=esp.content)
  #print(esp)

  ingles = llm.invoke(espannol)
  print(ing)
  image = pipe(ing).images[0]

  display(image)
  return image

generador_imagenes.invoke("nave espacial cerca de jupiter")

In [ ]:
generador_imagenes.invoke("pizza en el espacio")



Instanciar LLM y unir Tool

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model = "llama3-groq-70b-8192-tool-use-preview",
    temperature = 0.0
)

tools = [generador_imagenes]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
llm = ChatGroq(
    model="llama3-groq-70b-8192-tool-use-preview", api_key=userdata.get('GROQ_API_KEY'),
    temperature=0.3
)

tools = [generador_imagenes]

llm_with_tools = llm.bind_tools(tools)

Configurar el chatbot para que tenga memoria

In [ ]:
from langchain.tools import BaseTool
from langchain.agents import AgentExecutor, Tool

MEMORY_KEY = "chat_history"

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un asistente IA cuyo primer mensaje pregunta el nombre del usuario y avisa la ejecución de alguna herramienta",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

chat_history=[]


agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)


Correr chatbot

In [ ]:
while True:
  query = input("Usuario: ")
  if query.lower() in ["salir", "exit", "chao"]:
    break

  answer = agent_executor.invoke({"input": query, "chat_history": chat_history})
  chat_history.extend(
      [
          HumanMessage(content=query),
          AIMessage(content=answer["output"]),
      ]
  )

  print(f'IA: {answer["output"]}')

In [ ]:
agent_executor.invoke({"input": "Generar imagen caballo volador", "chat_history": chat_history})